In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
# 손실 함수를 초기화합니다.
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312774  [    0/60000]
loss: 2.292326  [ 6400/60000]
loss: 2.274190  [12800/60000]
loss: 2.263611  [19200/60000]
loss: 2.256120  [25600/60000]
loss: 2.224268  [32000/60000]
loss: 2.232446  [38400/60000]
loss: 2.198591  [44800/60000]
loss: 2.191319  [51200/60000]
loss: 2.151724  [57600/60000]
Test Error: 
 Accuracy: 35.6%, Avg loss: 2.152478 

Epoch 2
-------------------------------
loss: 2.165565  [    0/60000]
loss: 2.152468  [ 6400/60000]
loss: 2.095712  [12800/60000]
loss: 2.116814  [19200/60000]
loss: 2.073346  [25600/60000]
loss: 1.999505  [32000/60000]
loss: 2.041453  [38400/60000]
loss: 1.953708  [44800/60000]
loss: 1.954206  [51200/60000]
loss: 1.885826  [57600/60000]
Test Error: 
 Accuracy: 51.8%, Avg loss: 1.885446 

Epoch 3
-------------------------------
loss: 1.916484  [    0/60000]
loss: 1.885411  [ 6400/60000]
loss: 1.773611  [12800/60000]
loss: 1.824468  [19200/60000]
loss: 1.712748  [25600/60000]
loss: 1.654514  [32000/600